In [1]:
import os, sys
sys.path.append('../')
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import torch
import torchvision
import torchvision.transforms as transforms
from datasets.coco import CocoDataset
from utils.transforms import PhotometricDistort, Flip, Zoomout, \
                             ImageToTensor, CategoryToTensor, \
                             BoxToTensor, Resize, Normalize, \
                             CocoBoxToFracBoundaryBox, AssignObjToPbox
from functools import partial
import torch.nn as nn
import torch.nn.functional as F
from model.detector import SSD300
from loss.multibox import Loss
from torch.utils.tensorboard import SummaryWriter

DEVICE = 'cpu' if torch.cuda.device_count() == 0 else 'cuda:0'
print(DEVICE)

/Users/jcheng/opt/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


cpu


In [2]:
# initialize network 
N_CLASSES = 81
ssd = SSD300(n_classes=N_CLASSES, device=DEVICE)

In [3]:
# transforms
IMG_SZ = 300

distort_tfms  = [PhotometricDistort(0.5),
                 Flip(0.5)]
zoom_tfms     = [Zoomout(0.2, max_scale=2.5)]
toTensor_tfms = [ImageToTensor(), 
                 CategoryToTensor(), 
                 BoxToTensor()]
resize_tfms   = [Resize((IMG_SZ, IMG_SZ))]
target_tfms   = [CocoBoxToFracBoundaryBox(),
                 AssignObjToPbox(pboxes=ssd.pboxes, threshold=0.5)]
imgNorm_tfms  = [Normalize()]

tfms = transforms.Compose(distort_tfms + toTensor_tfms + zoom_tfms + imgNorm_tfms + resize_tfms + target_tfms)

# create dataset & dataloader
ds = CocoDataset('../datasets/', 'samples', 'instances', transforms=tfms)
assert ssd.n_classes == len(ds.id2cat.keys()), "# of class mismatch between dataset and ssd"
dl = torch.utils.data.DataLoader(ds, batch_size=4, shuffle=True, num_workers=2,
                                 collate_fn=partial(ds.collate_fn, img_resized=True))

# create tensorboard summary writer
writer = SummaryWriter('runs/vgg_base')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [4]:
# loss function
loss = Loss(neg_pos_ratio=3, 
            alpha=1.,
            device=DEVICE)

In [5]:
# get trainable biases and params
trainable_biases = list()
trainable_params = list()
for name, param in ssd.named_parameters():
    if param.requires_grad:
        if name.endswith('.bias'): trainable_biases.append(param)
        else: trainable_params.append(param)

In [6]:
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
# init optimizer
opt = torch.optim.SGD(params=[{'params': trainable_biases}, 
                              {'params': trainable_params}],
                      lr=1e-3, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

In [7]:
# test 1 batch of forward and backprop
for i, batch in enumerate(dl):
    # zero grads
    opt.zero_grad()    
    # forward pass
    images = batch['images']
    pred_boxes, pred_scores = ssd(images)
    # compute loss
    true_locs, true_cls = batch['pbox_offsets'], batch['pbox_classes']
    cost = loss(pred_boxes, pred_scores, true_locs, true_cls)
    # back prop
    cost.backward()
    # update
    opt.step()
    print(f'batch {i} completed')

/Users/jcheng/opt/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


batch 0 completed
batch 1 completed
batch 2 completed
